In [1]:
import sys, os, datetime
sys.path.append("../../../")

from utils.contract_utils import init_contract
from utils.network_utils import configure_network_and_connect

import brownie

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_style("whitegrid")

In [3]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network_and_connect(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

In [4]:
pools = {
    "ankreth": "0xA96A65c051bF88B4095Ee1f2451C2A9d43F53Ae2",
    "reth": "0xF9440930043eb3997fc70e1339dBb11F341de7A8",
    "alusd": "0x43b4FdFD4Ff969587185cDB6f0BD875c5Fc83f8c",
    "frax": "0xd632f22692FaC7611d2AA1C0D552930D43CAEd3B",
}

In [5]:
pools = {
    "ren": "0x93054188d876f558f4a66B2EF1d97d16eDf0895B",
    "bbtc": "0x071c661B4DeefB59E2a3DdB20Db036821eeE8F4b",
    "sbtc": "0x7fC77b5c7614E1533320Ea6DDc2Eb61fa00A9714",
    "hbtc": "0x4CA9b3063Ec5866A4B82E437059D2C43d1be596F",
    "tbtc": "0xC25099792E9349C7DD09759744ea681C7de2cb66",
    "pbtc": "0x7F55DDe206dbAD629C080068923b36fe9D6bDBeF",
    "obtc": "0x7F55DDe206dbAD629C080068923b36fe9D6bDBeF"
}

In [6]:
pools = {
    "usdm": "0x5B3b5DF2BF2B6543f78e053bD91C4Bdd820929f1",
}

In [12]:
block_number_start = brownie.web3.eth.block_number - 50000

In [13]:
def get_token_balances_from_block(
    block_number_start: int,
    pool_contract: brownie.Contract, 
    block_steps: int = 10000
) -> pd.DataFrame:

    current_block = brownie.web3.eth.block_number
    blocks_to_calculate = list(range(block_number_start, current_block, block_steps))
    blocks_to_calculate.append(current_block)

    data = {'timestamp': [], 'block_number': [], 'total_coin_balances': []}

    for block_number in blocks_to_calculate:

        block_details = brownie.web3.eth.get_block(block_number)
        block_time = datetime.datetime.fromtimestamp(block_details['timestamp'])

        try:
            pool_balances = pool_contract.get_balances(block_identifier=block_number)
        except ValueError:
            continue
        except:  # old vyper contracts did not have get_balances method
            pool_balances = []
            for coin_index in range(10):
                try:
                    pool_balance_coin = pool_contract.balances(coin_index, block_identifier=block_number)
                    pool_balances.append(pool_balance_coin)
                except ValueError:
                    break

        total_coins = sum(pool_balances)

        data['timestamp'].append(block_time)
        data['block_number'].append(block_number)
        data['total_coin_balances'].append(total_coins)

    df_data = pd.DataFrame(data)

    return df_data.set_index('timestamp')


In [ ]:
dict_data_pools = {}
for pool_name, pool_addr in pools.items():

    pool_contract_obj = init_contract(pool_addr)

    dict_data_pools[pool_name] = get_token_balances_from_block(
        block_number_start=block_number_start,
        pool_contract=pool_contract_obj,
        block_steps=200
    )
dict_data_pools

In [ ]:
bribe_tx = pd.Timestamp("2021-11-11 00:35:00", tz="utc")
bribe_tx

In [ ]:
for pool_name, df_pool_tvl in dict_data_pools.items():

    fig, ax = plt.subplots(1,1, figsize=(25, 8))

    plot_start_date = pd.Timestamp("2021-09-01")
    df_plot = df_pool_tvl[df_pool_tvl.index > plot_start_date]
    df_plot.reset_index(inplace=True)

    decimals = 1e-18
    millionth_decimal = False
    if (df_plot.total_coin_balances * 1e-18).mean() > 1e8:
        decimals *= 1e-7
        millionth_decimal = True

    ax.plot(df_plot.timestamp, df_plot.total_coin_balances * decimals, marker='x', color='r', lw=3, ms=10, label=pool_name)
    ax.axvline(x=bribe_tx, label="Andre's Tweet", color='k', ls='--')

    ax.set_ylabel("")
    if not millionth_decimal:
        ax.set_ylabel("Total Coins", fontsize=25)
    else:
        ax.set_ylabel("Total Coins [Millions]", fontsize=25)
    ax.set_title(f"USDM Curve Pool", fontsize=25);
    ax.tick_params(axis='both', which='major', labelsize=25);

    ax.legend(fontsize=25);
    try:

        ax.ticklabel_format(useOffset=False, style='plain');
    
    except AttributeError:

        continue